In [15]:
import pandas as pd
import json
import en_core_web_sm
import spacy
import gensim.parsing.preprocessing
import pprint

ModuleNotFoundError: No module named 'gensim'

In [16]:
input_path = '../../data/data_2021-02-01 22:27:13.862993.json'

In [20]:
with open(input_path, encoding="utf-8") as f:
    data = json.load(f)
# Keywords from every paper  
keywords_df = pd.json_normalize(data['papers'])['keywords']

print("No of papers: {}".format(keywords_df.shape[0]))

AttributeError: module 'pandas' has no attribute 'json_normalize'

In [18]:
STOPWORDS = nlp.Defaults.stop_words
nlp = spacy.load("en_core_web_sm")

NameError: name 'nlp' is not defined

In [19]:
def process(flat_words, lemmatize=True, min_word_len=2):
  flat_words = [gensim.parsing.preprocessing.strip_non_alphanum(word) for word in flat_words]
  doc = spacy.tokens.Doc(nlp.vocab, words=flat_words)

  processed = []

  if lemmatize:
    processed = [token.lemma_ for token in doc]
    processed = [token.lower() for token in processed]
  else:
    processed = [token.lower_ for token in doc]

  processed = [token for token in processed if token not in STOPWORDS]
  processed = [token for token in processed if len(token) >= min_word_len]

  return processed


In [25]:
def process_corpus(keywords_df, lemmatize=True, min_word_len=2):
  processed_corpus = []
  flat_words = []

  # n: no of research papers
  # [[kw11, kw12, ...] ...[kwn1, kwn2, kwn3, ...]]
  keywords = keywords_df.tolist()
  # j: no of document in which the keyword is present
  # kwji -> [wji1, wij2, ...] ~ list of all words associated with keyword i of paper j
  # [kwj1, kwj2, ...] -> [kwj11, kwj12, wkwj13, kwj21, kwj22, ...] ~ list of all words associated with the keywords of paper j
  for l in keywords:
    tmp = []
    for keyword in l:
      words = keyword.split(" ")
      for word in words:
        tmp.append(word)
    flat_words.append(tmp)

  for l in flat_words:
    processed_corpus.append(process(l, True, 2))

  return processed_corpus


In [79]:
from gensim import corpora

processed_corpus = process_corpus(keywords_df)
dictionary = corpora.Dictionary(processed_corpus)

bow_corpus = [dictionary.doc2bow(words) for words in processed_corpus]


2230


In [47]:
from gensim import models

# Train the model
tfidf = models.TfidfModel(bow_corpus)

In [48]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=len(dictionary.token2id))

In [87]:
from collections import defaultdict

def compute_similarities(processed_corpus, threshold=0.9):
  similarities = defaultdict(list)

  for query_doc_index in range(len(processed_corpus)):
    query_doc =  processed_corpus[query_doc_index]
    query_bow = dictionary.doc2bow(query_doc)
    sims = index[tfidf[query_bow]]
    for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
      if score >= threshold and document_number != query_doc_index:
        similarities[query_doc_index].append((document_number, score))

  return similarities


In [88]:
sims = compute_similarities(processed_corpus)

In [ ]:
sims

Brainstorming with Chen from 05.February.2021: 

Paper j: kwj1 -> kwi1,......,n 
0 - 1 - save only min 

keywords processing 
remove learning? -> outliers outliers outliers!

1. Keine andere Distanc (word2vec nicht ausprobieren) [Chris langweilt sich -> word2vec] aber wir bleiben bei tfidf
2. Wie viele Cluster bekommen wir mit diesem BOW/TFIDF Ansatz? (threshold (?)) -> 
3. Clustern auf die Similarities -> Neue Daten mit einem speziellen Label 
